# Import Gene Annotation File

https://github.com/hakha-most/gwas_eqtl/blob/master/gene_annotations/genes.protein_coding.v39.gtf

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import os 
os.chdir("/Users/tanvibansal/Documents/GitHub/Capstone")
# Local path to GTF file
gtf_file_path = 'genes.protein_coding.v39.gtf'

# Load the GTF file into a pandas DataFrame and use the first row as headers
gtf_df = pd.read_csv(gtf_file_path, sep='\t', header=0)

# Display the first few rows of the dataframe
gtf_df.sort_values('start')
gtf_df.head()

,chr,start,end,strand,GeneSymbol,cons,gene,hgnc_id,tss,tes
0,chr1,65419,71585,+,ENSG00000186092,protein_coding,OR4F5,HGNC:14825,65419,71585
1,chr1,621096,622034,-,ENSG00000284662,protein_coding,OR4F16,HGNC:15079,622034,621096
2,chr1,859303,879955,+,ENSG00000187634,protein_coding,SAMD11,HGNC:28706,859303,879955
3,chr1,879583,894689,-,ENSG00000188976,protein_coding,NOC2L,HGNC:24517,894689,879583
4,chr1,895964,901099,+,ENSG00000187961,protein_coding,KLHL17,HGNC:24023,895964,901099


#### convert chr to int

In [2]:
gtf_df['chr'] = gtf_df['chr'].str[3:].astype(int)
gtf_df['start'] = gtf_df['start'].astype(int)
gtf_df.groupby('chr')['start'].agg({'min', 'max'}).reset_index()

,chr,max,min
0,1,249200395,65419
1,2,242836139,38814
2,3,197687071,238446
3,4,190945522,53180
4,5,180794269,92283
5,6,170884383,291630
6,7,158820866,330136
7,8,146277851,116086
8,9,140772234,14475
9,10,135437399,92828


Positions reset for each chromosome

### File Description

This file contains **gene annotations** for **protein-coding genes** and provides information about the **location of genes on the genome**. Below is a description of each column:

1. **chr**: The chromosome where the gene is located (e.g., `chr1`, `chr2`, etc.).
2. **start**: The start position of the gene on the chromosome (in base pairs).
3. **end**: The end position of the gene on the chromosome (in base pairs).
4. **strand**: Indicates the strand on which the gene is located (`+` for forward strand, `-` for reverse strand).
5. **GeneSymbol**: The symbol or name of the gene (e.g., `OR4F5`, `SAMD11`), typically assigned by organizations like HGNC (HUGO Gene Nomenclature Committee).
6. **cons**: This column indicates the type of gene. In this file, all genes are classified as **protein_coding**.
7. **gene**: The Ensembl gene ID, a unique identifier for the gene (e.g., `ENSG00000186092`).
8. **hgnc_id**: The unique identifier for the gene assigned by the **HGNC** (HUGO Gene Nomenclature Committee).
9. **tss**: The **transcription start site**, the position where transcription of the gene starts on the chromosome.
10. **tes**: The **transcription end site**, the position where transcription of the gene ends on the chromosome.

# Import GWAS data

 https://docs.google.com/spreadsheets/d/1kvPoupSzsSFBNSztMzl04xMoSC3Kcx3CrjVf4yBmESU/edit?gid=178908679#gid=178908679 (row 7217)
 
 50_irnt.gwas.imputed_v3.both_sexes.tsv.bgz' 

In [3]:
# Path to GWAS file
gwas_file_path = '50_irnt.gwas.imputed_v3.both_sexes.tsv.bgz'
import gzip 
# Open and read the compressed .bgz file using gzip
with gzip.open(gwas_file_path, 'rt') as f:
    # Load the file into a pandas DataFrame
    gwas_df = pd.read_csv(f, sep='\t')

In [4]:
# Display the first few rows to inspect the structure of the GWAS file
gwas_df

,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval
0,1:15791:C:T,T,5.440760e-09,True,360388,0.003922,0.003474,18.049900,178.468000,0.101138,0.919441
1,1:69487:G:A,A,5.761760e-06,True,360388,4.152940,-0.087536,-0.041345,0.349577,-0.118272,0.905852
2,1:69569:T:C,C,1.881580e-04,True,360388,135.620000,-2.079970,-0.046994,0.062695,-0.749571,0.453514
3,1:139853:C:T,T,5.674710e-06,True,360388,4.090200,-0.105659,-0.042056,0.349588,-0.120302,0.904244
4,1:692794:CA:C,C,1.106010e-01,False,360388,79718.500000,101.503000,0.000797,0.002899,0.274788,0.783479
...,...,...,...,...,...,...,...,...,...,...,...
13791462,X:154929412:C:T,T,2.454380e-01,False,360388,176906.000000,-787.373000,-0.001657,0.001588,-1.044020,0.296477
13791463,X:154929637:CT:C,C,2.297000e-01,False,360388,165562.000000,-734.113000,-0.001654,0.001657,-0.998454,0.318060
13791464,X:154929952:CAA:C,C,2.393990e-01,False,360388,172553.000000,-480.000000,-0.001689,0.001670,-1.011490,0.311783
13791465,X:154930230:A:G,G,2.458510e-01,False,360388,177204.000000,-706.111000,-0.001520,0.001587,-0.957369,0.338381


### GWAS Summary Statistics Dataset Description

1. **variant**: The unique identifier for each SNP (Single Nucleotide Polymorphism). This can include information like chromosome, position, reference allele, and alternative allele (e.g., `1:12345:A:G`).
   - **Example**: `1:10583:T:G`

2. **minor_allele**: The allele that is less frequent in the population (minor allele) for this particular SNP.
   - **Example**: `G`

3. **minor_AF**: The **minor allele frequency** (AF), which represents the frequency of the minor allele in the population. It ranges from 0 to 1.
   - **Example**: `0.35` (35% of individuals carry the minor allele)

4. **low_confidence_variant**: A flag indicating whether the variant has **low confidence** due to imputation quality or other uncertainties. Values may be `TRUE` or `FALSE`.
   - **Example**: `FALSE`

5. **n_complete_samples**: The number of samples for which complete genotype data is available for this variant.
   - **Example**: `300,000`

6. **AC**: The **allele count** of the minor allele, i.e., the number of times the minor allele appears in the study population (across all samples).
   - **Example**: `50000`

7. **ytx**: Likely a placeholder for a phenotype-related statistic; depending on the dataset, this could represent something like the trait mean or effect size (its exact meaning depends on the specific analysis).

8. **beta**: The **effect size** of the SNP on the trait being studied (in this case, likely height). It represents the change in the trait per additional copy of the minor allele.
   - **Example**: `0.05` (the trait increases by 0.05 units for each additional copy of the minor allele)

9. **se**: The **standard error** of the effect size (beta), indicating the precision of the estimated effect.
   - **Example**: `0.01`

10. **tstat**: The **t-statistic** for the beta estimate, which is the ratio of the beta estimate to its standard error.
    - **Example**: `5.0` (higher values indicate more significant associations)

11. **pval**: The **p-value** of the association between the SNP and the trait. This indicates the significance of the result, with smaller p-values suggesting stronger evidence that the SNP is associated with the trait.
    - **Example**: `1.2e-6` (a very small p-value, indicating strong evidence of association)

# Find Variant Chromosome and Position

In [5]:
gwas_df[['chr', 'pos', 'ref', 'alt']] = gwas_df['variant'].str.split(':', expand=True)
gwas_df = gwas_df.loc[gwas_df['chr'] != 'X']
gwas_df['chr'] = gwas_df['chr'].astype(int)
gwas_df['pos'] = gwas_df['pos'].astype(int)

gwas_df.groupby('chr')['pos'].agg({'min','max'}).reset_index()

/var/folders/z0/1hhlpnfn2mz3vk7t57zdcy0w0000gn/T/ipykernel_46541/2974836220.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gwas_df['chr'] = gwas_df['chr'].astype(int)
/var/folders/z0/1hhlpnfn2mz3vk7t57zdcy0w0000gn/T/ipykernel_46541/2974836220.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gwas_df['pos'] = gwas_df['pos'].astype(int)


,chr,max,min
0,1,249230914,15791
1,2,243082521,10181
2,3,197900375,60197
3,4,190922257,13012
4,5,180736061,11882
5,6,170934414,202076
6,7,159128550,27916
7,8,146303560,34440
8,9,141104957,40997
9,10,135503523,61334


It appears that position resets on each chromosome 

# Join to Find 5 Nearest Genes 

In [ ]:
from pandarallel import pandarallel
import os
import gc
os.chdir("/Users/tanvibansal/Documents/GitHub/Capstone")
#grab a small sample of the gwas dataframe containing mutation information 
gwas_df.head()
gtf_df.head()
gwas_sample = gwas_df.iloc[np.random.choice(range(len(gwas_df)),size=100000,replace=False)] 

#drop the genes that are not present in the s_het file as these may cause na's that will break the ldsc algorithm in the next step
s_het_info = pd.read_excel('s_het_info.xlsx',sheet_name=1).rename(columns={"ensg":"GeneSymbol"}).set_index("GeneSymbol")
gtf_df_tidy = s_het_info.join(gtf_df.set_index("GeneSymbol"),how="inner")[["chr","start","end","gene"]]

#write a function to find the 5 nearest genes by absolute difference in mutation position and gene start position
def nearest_gene_finder(m):
    #get the absolute l1 distance 
    dists = np.abs(gtf_df_tidy.loc[gtf_df_tidy.chr == m.chr,"start"] - m.pos)
    top_5 = dists.sort_values()[0:5]
    out = pd.DataFrame(m).T
    out[["GeneSymbol.f1","GeneSymbol.f2","GeneSymbol.f3","GeneSymbol.f4","GeneSymbol.f5"]] = top_5.index
    out[["dist.f1","dist.f2","dist.f3","dist.f4","dist.f5"]] = top_5.values/1000 #convert distance to kilobases by dividing through 1000
    return(out)

#create a sample of the gwas dataframe with only mutations on chr 22 for this stage
gwas_sample_22 = gwas_df.loc[gwas_df.chr ==22]

#apply the function over each row of the sample mutation df 
pandarallel.initialize(progress_bar=True)
mutn_5_nearest_genes_sample = gwas_sample_22.parallel_apply(lambda m: nearest_gene_finder(m),axis=1)
gc.collect()
mutn_5_nearest_genes_sample_df.loc[mutn_5_nearest_genes_sample_df['dist.f1'] == 0,"dist.f1"] = 1/1000
mutn_5_nearest_genes_sample_df = pd.concat(list(mutn_5_nearest_genes_sample))

In [19]:
#now join the s_het values onto each of our 5 nearest genes and compute the distance weighted s_het 
s_het_w = mutn_5_nearest_genes_sample_df.copy(deep=True)
for i in range(1,6):
    s_het_temp = s_het_info[["post_mean"]].reset_index().rename(columns = {"GeneSymbol":"GeneSymbol.f%s"%(i)}).set_index("GeneSymbol.f%s"%(i))
    s_het_w = s_het_w.reset_index().set_index("GeneSymbol.f%s"%(i)).join(s_het_temp)
    s_het_w["s_het.f%s"%(i)] = s_het_w["post_mean"]
    s_het_w["s_het_w.f%s"%(i)] = s_het_w["post_mean"]/s_het_w["dist.f%s"%(i)]
    s_het_w = s_het_w.drop(columns="post_mean")
s_het_w = s_het_w.reset_index() 
s_het_w

,GeneSymbol.f5,GeneSymbol.f4,GeneSymbol.f3,GeneSymbol.f2,GeneSymbol.f1,index,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,chr,pos,ref,alt,dist.f1,dist.f2,dist.f3,dist.f4,dist.f5,s_het.f1,s_het_w.f1,s_het.f2,s_het_w.f2,s_het.f3,s_het_w.f3,s_het.f4,s_het_w.f4,s_het.f5,s_het_w.f5
0,ENSG00000183307,ENSG00000177663,ENSG00000215568,ENSG00000172967,ENSG00000198445,13184537,22:16464274:A:C,C,0.084046,False,360388,60578.3,-262.512,0.003413,0.003183,1.07204,0.283702,22,16464274,A,C,607.367,800.028,978.552,1101.570,1132.913,0.002707,0.000004,0.001343,0.000002,0.000249,2.546518e-07,0.008926,0.000008,0.118445,0.000105
1,ENSG00000183307,ENSG00000177663,ENSG00000215568,ENSG00000172967,ENSG00000198445,13184538,22:16488635:C:A,A,0.080987,False,360388,58373.6,-220.884,0.003806,0.003111,1.22341,0.221175,22,16488635,C,A,583.006,775.667,954.191,1077.209,1108.552,0.002707,0.000005,0.001343,0.000002,0.000249,2.611532e-07,0.008926,0.000008,0.118445,0.000107
2,ENSG00000183307,ENSG00000177663,ENSG00000215568,ENSG00000172967,ENSG00000198445,13184539,22:16488702:G:C,C,0.080078,False,360388,57718.1,-250.356,0.003739,0.003121,1.19791,0.230954,22,16488702,G,C,582.939,775.600,954.124,1077.142,1108.485,0.002707,0.000005,0.001343,0.000002,0.000249,2.611715e-07,0.008926,0.000008,0.118445,0.000107
3,ENSG00000183307,ENSG00000177663,ENSG00000215568,ENSG00000172967,ENSG00000198445,13184540,22:16495833:C:A,A,0.078102,False,360388,56294.0,-249.417,0.004095,0.003069,1.33402,0.182199,22,16495833,C,A,575.808,768.469,946.993,1070.011,1101.354,0.002707,0.000005,0.001343,0.000002,0.000249,2.631382e-07,0.008926,0.000008,0.118445,0.000108
4,ENSG00000183307,ENSG00000177663,ENSG00000215568,ENSG00000172967,ENSG00000198445,13184541,22:16498458:G:A,A,0.07444,False,360388,53654.4,-203.467,0.004117,0.003224,1.27711,0.201566,22,16498458,G,A,573.183,765.844,944.368,1067.386,1098.729,0.002707,0.000005,0.001343,0.000002,0.000249,2.638696e-07,0.008926,0.000008,0.118445,0.000108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179761,ENSG00000008735,ENSG00000100299,ENSG00000251322,ENSG00000100312,ENSG00000079974,13364298,22:51229717:A:T,T,0.29612,False,360388,213436.0,-437.663,-0.002442,0.002004,-1.21861,0.222992,22,51229717,A,T,23.788,53.093,116.874,168.535,190.495,0.003011,0.000127,0.008019,0.000151,0.248066,2.122508e-03,0.001543,0.000009,0.706036,0.003706
179762,ENSG00000008735,ENSG00000100299,ENSG00000251322,ENSG00000100312,ENSG00000079974,13364299,22:51229805:T:C,C,0.072999,False,360388,52615.7,-172.528,0.001794,0.003184,0.563533,0.573073,22,51229805,T,C,23.876,53.181,116.962,168.623,190.583,0.003011,0.000126,0.008019,0.000151,0.248066,2.120911e-03,0.001543,0.000009,0.706036,0.003705
179763,ENSG00000008735,ENSG00000100299,ENSG00000251322,ENSG00000100312,ENSG00000079974,13364300,22:51231220:A:G,G,0.053676,False,360388,38688.2,-20.4974,0.002084,0.003916,0.532177,0.594604,22,51231220,A,G,25.291,54.596,118.377,170.038,191.998,0.003011,0.000119,0.008019,0.000147,0.248066,2.095559e-03,0.001543,0.000009,0.706036,0.003677
179764,ENSG00000008735,ENSG00000100299,ENSG00000251322,ENSG00000100312,ENSG00000079974,13364301,22:51237063:T:C,C,0.299014,False,360388,215522.0,-260.204,-0.001741,0.001944,-0.895286,0.370635,22,51237063,T,C,31.134,60.439,124.220,175.881,197.841,0.003011,0.000097,0.008019,0.000133,0.248066,1.996989e-03,0.001543,0.000009,0.706036,0.003569


## Prepare the annotation file containing our features for the LDSC program 

In [20]:
#format the dataframe's column names, columns to retain, and match the SNPs present in the bim file
s_het_w = s_het_w.reset_index().rename(columns={'chr': 'CHR','pos': 'BP','variant': 'SNP'})
annot_df = s_het_w[['CHR', 'BP', 'SNP', 's_het_w.f1', 's_het_w.f2', 's_het_w.f3', 's_het_w.f4', 's_het_w.f5']]
#annot_df = s_het_w[['CHR', 'BP', 'SNP', 's_het.f1', 's_het.f2', 's_het.f3', 's_het.f4', 's_het.f5']]
annot_df = annot_df.sort_values(['CHR', 'BP'])
annot_df

,CHR,BP,SNP,s_het_w.f1,s_het_w.f2,s_het_w.f3,s_het_w.f4,s_het_w.f5
0,22,16464274,22:16464274:A:C,0.000004,0.000002,2.546518e-07,0.000008,0.000105
1,22,16488635,22:16488635:C:A,0.000005,0.000002,2.611532e-07,0.000008,0.000107
2,22,16488702,22:16488702:G:C,0.000005,0.000002,2.611715e-07,0.000008,0.000107
3,22,16495833,22:16495833:C:A,0.000005,0.000002,2.631382e-07,0.000008,0.000108
4,22,16498458,22:16498458:G:A,0.000005,0.000002,2.638696e-07,0.000008,0.000108
...,...,...,...,...,...,...,...,...
179761,22,51229717,22:51229717:A:T,0.000127,0.000151,2.122508e-03,0.000009,0.003706
179762,22,51229805,22:51229805:T:C,0.000126,0.000151,2.120911e-03,0.000009,0.003705
179763,22,51231220,22:51231220:A:G,0.000119,0.000147,2.095559e-03,0.000009,0.003677
179764,22,51237063,22:51237063:T:C,0.000097,0.000133,1.996989e-03,0.000009,0.003569


In [238]:
#column names for the bim file according to the plink documentation site
names = ["CHR","SNP","POS","BP","A1","A2"]
os.chdir("/Users/tanvibansal/Documents/GitHub/ldsc")
bim = pd.read_csv("22.bim",sep="\t",header=None,names=names)

#join the annotated df onto the bim file and sort in the order of the bim file exactly to prep for ldsc. we should have the same exact # of rows 
annot_df_filt = bim.set_index(["CHR","BP"]).join(annot_df.set_index(["CHR","BP"]),how="left",rsuffix=".a").drop_duplicates(subset=["SNP"])
annot_df_sort = annot_df_filt.reset_index().set_index("SNP").loc[bim.SNP.values]
annot_df_sort["CM"] = 0.0

#select the cols of interest and write the file out as .annot to use for ldsc
#annot_df_prep = annot_df_sort.reset_index()[["CHR","BP","SNP", "CM", "s_het_w.f1", "s_het_w.f2", "s_het_w.f3", "s_het_w.f4", "s_het_w.f5"]]
annot_df_prep = annot_df_sort.reset_index()[["CHR","BP","SNP", "CM", "s_het.f1", "s_het.f2", "s_het.f3", "s_het.f4", "s_het.f5"]]
annot_df_prep = annot_df_prep.dropna()
annot_df_prep.to_csv("1kg_eur/s_het_unweighted_ch22.annot",sep="\t",index=False)
annot_df_prep

#unique_snps = annot_df_prep['SNP'].unique()  # Get unique SNPs to avoid duplicates

# Save the SNPs to a text file
#with open('1kg_eur/s_het_22/filtered_snps.txt', 'w') as f:
 #   for snp in unique_snps:
  #      f.write(f"{snp}\n") 

NameError: name 'annot_df' is not defined

#### we need to run the following 2 commands to get the ldsc computation to run as intended 

1. Make the .bed/.bim files with our filtered SNPs


plink --bfile /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/22 --extract /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/s_het_22/filtered_snps.txt --make-bed --out /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/s_het_22

2. Compute the ld scores with our inputs specified

python ldsc.py --l2 --bfile /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/s_het_22_uw/s_het_22_uw --ld-wind-cm 1 --annot /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/s_het_22_uw/s_het_uw_ch22.annot --out 1kg_eur/s_het_22_uw/s_het_22_uw

In [198]:
import gzip 
with gzip.open('1kg_eur/s_het_22.l2.ldscore.gz', 'rb') as f: 
    file_content = pd.read_csv(f,sep="\t") 


## Generate plink triplet files needed for LD score computation for all chromosomes

#### 1. Convert and filter 10000 genomes data in .vcf format to plink triplet files for europeans

##### 1.1 Extract IDs for european individuals in the 1000 genomes data set 

In [129]:
#download file containing metadata and filter for EUR super population 
int_call_samples_meta = pd.read_csv("/Users/tanvibansal/Documents/GitHub/Capstone/integrated_call_samples_v3.20130502.ALL.panel.txt",sep="\t")
european_individuals = int_call_samples_meta.loc[int_call_samples_meta.super_pop == "EUR"]
european_individuals

#download file containing Family ID and Individual ID for all individuals, filter for individual ID in european_individuals metadata 
int_call_samples_individuals = pd.read_csv("/Users/tanvibansal/Documents/GitHub/Capstone/integrated_call_samples_v3.20200731.ALL.ped.txt",sep="\t")
european_individuals_ids = int_call_samples_individuals.iloc[np.isin(int_call_samples_individuals['Individual ID'].values, european_individuals['sample'].values)]
eur_individuals_ids = european_individuals_ids[["Family ID","Individual ID"]]
print(eur_individuals_ids.to_markdown())

#write file containing family and individual IDs to .txt file to be used in the filtering step next
eur_individuals_ids.to_csv("/Users/tanvibansal/Documents/GitHub/Capstone/eur_individuals.txt",sep="\t",index=False)

|      | Family ID   | Individual ID   |
|-----:|:------------|:----------------|
|    0 | HG00096     | HG00096         |
|    1 | HG00097     | HG00097         |
|    3 | HG00099     | HG00099         |
|    4 | HG00100     | HG00100         |
|    5 | HG00101     | HG00101         |
|    6 | HG00102     | HG00102         |
|    7 | HG00103     | HG00103         |
|    9 | HG00105     | HG00105         |
|   10 | HG00106     | HG00106         |
|   11 | HG00107     | HG00107         |
|   12 | HG00108     | HG00108         |
|   13 | HG00109     | HG00109         |
|   14 | HG00110     | HG00110         |
|   15 | HG00111     | HG00111         |
|   16 | HG00112     | HG00112         |
|   17 | HG00113     | HG00113         |
|   18 | HG00114     | HG00114         |
|   19 | HG00115     | HG00115         |
|   20 | HG00116     | HG00116         |
|   21 | HG00117     | HG00117         |
|   22 | HG00118     | HG00118         |
|   23 | HG00119     | HG00119         |
|   24 | HG00120

##### 1.2 Run this command to get the plink triplet files from the 1000 genomes data for chromosome 22 filtered for european individuals

plink --vcf /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz --keep /Users/tanvibansal/Documents/GitHub/Capstone/eur_individuals.txt --make-bed --out /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/22_1000g

In [27]:
#read and display our generated .bim file 
bim = pd.read_csv("/Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur/22_1000g.bim",sep="\t",header=None,names=["CHR","SNP","CM","POS","A1","A2"])
bim = bim.astype({"POS":"int32"})
bim

,CHR,SNP,CM,POS,A1,A2
0,22,.,0,16050075,G,A
1,22,.,0,16050115,A,G
2,22,.,0,16050213,T,C
3,22,.,0,16050319,T,C
4,22,.,0,16050527,A,C
...,...,...,...,...,...,...
1103542,22,.,0,51241342,A,C
1103543,22,.,0,51241386,G,C
1103544,22,.,0,51244163,G,A
1103545,22,.,0,51244205,T,C


#### 2. SNP match 1000 genomes to GWAS and assign SNP IDs

The 1000 genomes data we generated the .bim file does not include SNP ID's, so we need to create a process here to add them to the .bim file so we can filter by ID's in the HapMap3 list. But, sometimes our generated .bim file has multiple records for mutations at a given base pair location/chromosome to keep track of multiple versions of a mutation at that location (see cell below). The GWAS correlations are computed only for specific versions of these mutations (i.e. Allele 1 = X -> Allele 1 = Y), so we'll need to filter the SNP's in the .bim file to match the SNP's in the GWAS df. 

The tricky part is that A1 and A2 may be swapped as different sources prefer different standards with minor allele assignment, so we'll need to find cases where SNP's in the .bim file match the chromosome and position of SNP's in the GWAS file, then further downselect to cases where A1 = A1 and A2 = A2 OR  A1 = A2 and A2 = A1. Then we can name the SNPs in accordance with the GWAS file, generate random ID's for SNP's without a matching GWAS ID, and output a valid .bim file format to regenerate the plink triplet.

In [28]:
#download modified GWAS file with HM's modifications to naming convention 
#gwas_imputed = pd.read_csv("/Users/tanvibansal/Documents/GitHub/Capstone/HEIGHT_irnt.gwas.imputed_v3.both_sexes.tsv",sep="\t")
gwas_df = gwas_df.rename(columns={"chr":"CHR","pos":"POS"})

#filter for chromosome of interest and cast position column to integer
gwas_22 = gwas_df.loc[(gwas_df.CHR ==22)]
gwas_22 = gwas_22.astype({"POS":"int32"})

#now join the 1000g generated bim file to the modified gwas file
bim_gwas_df = bim.merge(gwas_22, on = ["CHR","POS"],suffixes=(".bim",".gwas"),how="left")
bim_gwas_df

,CHR,SNP,CM,POS,A1,A2,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,ref,alt
0,22,.,0,16050075,G,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22,.,0,16050115,A,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22,.,0,16050213,T,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,.,0,16050319,T,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,.,0,16050527,A,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103873,22,.,0,51241342,A,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103874,22,.,0,51241386,G,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103875,22,.,0,51244163,G,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103876,22,.,0,51244205,T,C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#start with the subset of our .bim dataframe that has matching GWAS data populated. The rest of the df will get random ID's generated, we'll come back to that later
bim_gwas_df_no_null = bim_gwas_df.loc[~bim_gwas_df.ref.isnull()]

#get positions of SNPs that are duplicated in the non-null df. These SNP's will need to be filtered based on true allele matches in GWAS df
duplicate_snp_pos = bim_gwas_df_no_null.loc[bim_gwas_df_no_null.POS.duplicated(),"POS"].values
def duplicate_variant_finder(df):
    duplicate_variants = collections.Counter(bim_gwas_df_allele_match.variant.values) 
    dvs = [k  for k in duplicate_variants.keys() if duplicate_variants[k] > 1]
    return(dvs)
    
bim_gwas_df_duplicate = bim_gwas_df_no_null.iloc[np.isin(bim_gwas_df_no_null.POS.values,duplicate_snp_pos)].sort_values(by = "POS")

#define function to return 1 if the alleles in .bim are the same as in gwas (regardless of order), 0 if not
import collections 
def allele_match_finder(x):
    bim_alleles = x[['A1','A2']]
    gwas_alleles = x[['ref','alt']]
    same = collections.Counter(bim_alleles) == collections.Counter(gwas_alleles)
    if same:
        out = 1
    else:
        out = 0
    return out
bim_gwas_df_duplicate["allele_match"] = bim_gwas_df_duplicate.apply(lambda x: allele_match_finder(x),axis=1) 

#that works, lets just run allele_match_finder over the whole non-null dataset for simplicity and ignore the rows with allele_match = 0. 
#we'll need to generate random IDs for these later
bim_gwas_df_no_null["allele_match"] = bim_gwas_df_no_null.apply(lambda x: allele_match_finder(x),axis=1) 
bim_gwas_df_no_null

/var/folders/z0/1hhlpnfn2mz3vk7t57zdcy0w0000gn/T/ipykernel_46541/3492587372.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bim_gwas_df_no_null["allele_match"] = bim_gwas_df_no_null.apply(lambda x: allele_match_finder(x),axis=1)


,CHR,SNP,CM,POS,A1,A2,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,ref,alt,allele_match
5419,22,.,0,16464274,C,A,22:16464274:A:C,C,0.084046,False,360388.0,60578.3,-262.5120,0.003413,0.003183,1.072040,0.283702,A,C,1
5834,22,.,0,16488635,A,C,22:16488635:C:A,A,0.080987,False,360388.0,58373.6,-220.8840,0.003806,0.003111,1.223410,0.221175,C,A,1
5837,22,.,0,16488702,C,G,22:16488702:G:C,C,0.080078,False,360388.0,57718.1,-250.3560,0.003739,0.003121,1.197910,0.230954,G,C,1
6013,22,.,0,16495833,A,C,22:16495833:C:A,A,0.078102,False,360388.0,56294.0,-249.4170,0.004095,0.003069,1.334020,0.182199,C,A,1
6069,22,.,0,16498458,A,G,22:16498458:G:A,A,0.074440,False,360388.0,53654.4,-203.4670,0.004117,0.003224,1.277110,0.201566,G,A,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103673,22,.,0,51229717,T,A,22:51229717:A:T,T,0.296120,False,360388.0,213436.0,-437.6630,-0.002442,0.002004,-1.218610,0.222992,A,T,1
1103679,22,.,0,51229805,C,T,22:51229805:T:C,C,0.072999,False,360388.0,52615.7,-172.5280,0.001794,0.003184,0.563533,0.573073,T,C,1
1103716,22,.,0,51231220,G,A,22:51231220:A:G,G,0.053676,False,360388.0,38688.2,-20.4974,0.002084,0.003916,0.532177,0.594604,A,G,1
1103804,22,.,0,51237063,C,T,22:51237063:T:C,C,0.299014,False,360388.0,215522.0,-260.2040,-0.001741,0.001944,-0.895286,0.370635,T,C,1


In [30]:
bim_gwas_df_duplicate

,CHR,SNP,CM,POS,A1,A2,variant,minor_allele,minor_AF,low_confidence_variant,n_complete_samples,AC,ytx,beta,se,tstat,pval,ref,alt,allele_match
19710,22,.,0,17102425,T,C,22:17102425:CCT:C,C,0.288595,False,360388.0,208013.00,-323.8390,0.000658,0.001835,0.358479,0.719985,CCT,C,0
19711,22,.,0,17102425,C,CCT,22:17102425:CCT:C,C,0.288595,False,360388.0,208013.00,-323.8390,0.000658,0.001835,0.358479,0.719985,CCT,C,1
20885,22,.,0,17136274,A,T,22:17136274:T:A,A,0.004696,False,360388.0,3384.72,-40.6283,0.001257,0.013476,0.093280,0.925681,T,A,1
20886,22,.,0,17136274,A,T,22:17136274:T:TA,TA,0.004837,False,360388.0,3486.29,-42.7700,0.001997,0.012902,0.154784,0.876991,T,TA,0
21099,22,.,0,17141436,CCGAACG,C,22:17141436:C:CCGAA,CCGAA,0.294499,False,360388.0,212268.00,-287.7320,0.000939,0.001826,0.514520,0.606889,C,CCGAA,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097697,22,.,0,51041570,G,C,22:51041570:C:G,G,0.046123,False,360388.0,33244.50,-121.2010,0.001189,0.003951,0.300844,0.763534,C,G,1
1099169,22,.,0,51085928,AAAAAG,A,22:51085928:AAAAAGAAAAG:A,A,0.100454,False,360388.0,72404.80,-180.4010,-0.001366,0.002859,-0.477722,0.632848,AAAAAGAAAAG,A,0
1099170,22,.,0,51085928,A,AAAAAGAAAAG,22:51085928:A:AAAAAG,AAAAAG,0.015195,False,360388.0,10952.40,-64.5595,-0.009348,0.006906,-1.353550,0.175882,A,AAAAAG,0
1099168,22,.,0,51085928,AAAAAG,A,22:51085928:A:AAAAAG,AAAAAG,0.015195,False,360388.0,10952.40,-64.5595,-0.009348,0.006906,-1.353550,0.175882,A,AAAAAG,1


In [31]:
bim_gwas_df_allele_match = bim_gwas_df_no_null.loc[bim_gwas_df_no_null.allele_match ==1] 

#now lets check for duplicate variant IDs in our remaining set as the plink extraction will fail if any SNP IDs are repeated
duplicate_variants = collections.Counter(bim_gwas_df_allele_match.variant.values) 
dvs = [k  for k in duplicate_variants.keys() if duplicate_variants[k] > 1]
duplicate_variants_df = bim_gwas_df_no_null_drop_dupl.iloc[np.isin(bim_gwas_df_no_null_drop_dupl.variant,dvs)]
print("Variant ID's with duplicates remaining: %s"%(len(dvs))) 
duplicate_variants_df.head(15)

#we can see that variants with 2 instances are all replicated because A1 and A2 in the .bim file are present in both orders. 
#lets just take the cases where A2 == ref for now
x = duplicate_variants_df.iloc[1]
def allele_order_finder(x):
    if x.A2 == x.ref:
        out = 1
    else:
        out = 0
    return out
bim_gwas_df_no_null["allele_ordered"] = bim_gwas_df_no_null.apply(lambda x: allele_order_finder(x),axis=1)
bim_gwas_df_no_null

NameError: name 'bim_gwas_df_no_null_drop_dupl' is not defined

In [32]:
bim.iloc[np.isin(bim.POS.values, 16464274)]

,CHR,SNP,CM,POS,A1,A2
5419,22,.,0,16464274,C,A


In [33]:
#now lets test again how many duplicate variant IDs we have
#bim_gwas_df_allele_match_ordered = bim_gwas_df_no_null.loc[(bim_gwas_df_no_null.allele_match ==1) & (bim_gwas_df_no_null.allele_ordered == 1)] 
#duplicate_variants = collections.Counter(bim_gwas_df_allele_match_ordered.variant.values)
#dvs = [k for k in duplicate_variants.keys() if duplicate_variants[k] > 1]
#print("Variant ID's with duplicates remaining: %s"%(len(dvs)))

#no more duplicate variant IDs, now dump this all into a function to run over each row of the .bim file we can parallelize
def SNP_id_finder(x, gwas_df):
    x_gwas_df = gwas_df.loc[(gwas_df.CHR == x.CHR) & (gwas_df.POS == x.POS)]
    n_matches = len(x_gwas_df)
    alternate_id = "%s:%s"%(x.POS,x.name)
    if n_matches == 0:
        return alternate_id
    else:
        #allel match check
        allele_match = [collections.Counter(x[['A1','A2']]) == collections.Counter(x_gwas_df.iloc[i][['ref','alt']]) for i in range(len(x_gwas_df))]
        if sum(allele_match) == 0:
            return alternate_id
        else:
            matches = x_gwas_df.loc[allele_match]
            #if len(matches) == 1:
             #   return matches.variant.values.item()
            #else:
            #allele order check
            allele_ordered = [x.A2 == matches.iloc[i]['ref'] for i in range(len(matches))]
            if sum(allele_ordered) == 0: # this should never happen but handling it here just in case
                return alternate_id
            else:
                matches_remaining = matches.loc[allele_ordered]
                if len(matches_remaining) == 1:
                    return matches_remaining.variant.values.item() 
                else:
                    return alternate_id # this should never happen but handling it here just in case

In [34]:
#run function over each row of the bim dataset
from pandarallel import pandarallel
import gc
pandarallel.initialize(progress_bar = False)
bim_ids = bim.parallel_apply(lambda x: SNP_id_finder(x, gwas_22),axis=1)
gc.collect()
duplicated_bim_ids = collections.Counter(bim_ids)
duplicate_ids = [k for k in duplicated_bim_ids.keys() if duplicated_bim_ids[k] > 1]
print("Duplicate SNP ID's: %s"%(len(duplicate_ids)))

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Duplicate SNP ID's: 0


In [101]:
#no duplicate ids, we're good to replace the SNP ID column with our computed IDs and write the whole modified .bim file out to txt
bim.loc[:,"SNP"]=bim_ids
bim.to_csv("/Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/modified.bim",sep="\t",header=None,index=False)

#find the common SNP ids between annot_df and .bim now and write out to .txt file to ensure we can run LDSC in the next stage
matches = bim_ids.apply(lambda x: len(x.split(":"))).sort_values(ascending=False)
matches = matches.loc[matches.values > 2]
unique_snps = bim_ids.loc[matches.index].to_frame().sort_values(by=0).rename(columns={0:"SNP"}).merge(annot_df.SNP).values
with open('/Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/filtered_snps.txt', 'w') as f:
    for snp in unique_snps:
        f.write(f"{snp[0]}\t") 

#filter annotation df for exact order and content match for SNP IDs and write to disk
annot_df_prep = annot_df.merge(bim_ids.loc[matches.index].to_frame(),left_on=["SNP"],right_on=[0]).sort_values(by="SNP").drop(columns=[0])
annot_df_prep["CM"] = 0.0 
annot_df_prep = annot_df_prep[["CHR","BP","SNP", "CM", "s_het_w.f1", "s_het_w.f2", "s_het_w.f3", "s_het_w.f4", "s_het_w.f5"]]
annot_df_prep.to_csv("/Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/s_het_weighted_ch22.annot",sep="\t",index=False)

##### Run this command to run the plink extraction
plink --bfile /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/22_1000g_modified --extract /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/filtered_snps.txt --make-bed --out /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/22_1000g_filt

plink --vcf /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz --keep /Users/tanvibansal/Documents/GitHub/Capstone/eur_individuals.txt --make-bed --out /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/22_1000g


plink --bfile /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur --extract /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/s_het_22/filtered_snps.txt --make-bed --out /Users/tanvibansal/Documents/GitHub/ldsc/1kg_eur/s_het_22

##### Run this comment to run the ld score computation 
python ldsc.py --l2 --bfile /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/22_1000g_filt --ld-wind-cm 1 --annot /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/s_het_weighted_ch22.annot --out /Users/tanvibansal/Documents/GitHub/Capstone/generated_plink_files/chr22/1000g_eur_hm3/s_het_w

In [ ]:
# Plot histograms for each difference column
def plot_histograms(df, columns):
    plt.figure(figsize=(14, 10))
    
    for i, col in enumerate(columns):
        plt.subplot(2, 3, i + 1)  # Create a subplot for each column
        df[col].hist(bins=50, color='skyblue', edgecolor='black')
        plt.title(f'Distribution of {col}')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

# Define the columns to plot
columns_to_plot = ['diff_f1', 'diff_f2', 'diff_f3', 'diff_b1', 'diff_b2', 'diff_b3']
# Call the function to plot histograms
plot_histograms(f_b_final_with_differences, columns_to_plot)

columns_to_plot = ["f1_dist","f2_dist","f3_dist","f4_dist","f5_dist"]
plot_histograms(nearest_genes_sample_df, columns_to_plot)

In [ ]:
s_het_info = pd.read_excel('s_het_info.xlsx',sheet_name=1)

In [40]:
# Function to perform the merge for all forward and backward GeneSymbols and rename the 'post_mean' columns
def merge_s_het_info_with_all(f_b_final_with_differences, s_het_info, num_merges=3):
    # Rename 'ensg' column in s_het_info to match the GeneSymbol columns in f_b_final_with_differences
    s_het_info = s_het_info.rename(columns={'ensg': 'GeneSymbol'})

    # Iterate over both forward and backward gene symbols
    for i in range(1, num_merges + 1):
        # Forward merge for each GeneSymbol_f
        f_b_final_with_differences = pd.merge(
            f_b_final_with_differences, 
            s_het_info[['GeneSymbol', 'post_mean']], 
            left_on=f'GeneSymbol_f{i}', 
            right_on='GeneSymbol', 
            how='left',
            suffixes=('', f'_f{i}')
        )
        
        # Rename the post_mean column to indicate the forward direction
        f_b_final_with_differences = f_b_final_with_differences.rename(
            columns={'post_mean': f's_het_post_f{i}'}
        )
        
        # Drop the duplicated 'GeneSymbol' column created during the merge
        f_b_final_with_differences = f_b_final_with_differences.drop(columns='GeneSymbol', errors='ignore')

        # Backward merge for each GeneSymbol_b
        f_b_final_with_differences = pd.merge(
            f_b_final_with_differences, 
            s_het_info[['GeneSymbol', 'post_mean']], 
            left_on=f'GeneSymbol_b{i}', 
            right_on='GeneSymbol', 
            how='left',
            suffixes=('', f'_b{i}')
        )
        
        # Rename the post_mean column to indicate the backward direction
        f_b_final_with_differences = f_b_final_with_differences.rename(
            columns={'post_mean': f's_het_post_b{i}'}
        )
        
        # Drop the duplicated 'GeneSymbol' column created during the merge
        f_b_final_with_differences = f_b_final_with_differences.drop(columns='GeneSymbol', errors='ignore')

    return f_b_final_with_differences

# Example usage:
f_b_final_merged_s_het = merge_s_het_info_with_all(f_b_final_with_differences, s_het_info, num_merges=3)

In [ ]:
f_b_final_merged_s_het

In [47]:
# f_b_final_merged_s_het[['diff_b1']] = abs(f_b_final_merged_s_het[['diff_b1']])
# f_b_final_merged_s_het[['diff_b2']] = abs(f_b_final_merged_s_het[['diff_b3']])
# f_b_final_merged_s_het[['diff_b3']] = abs(f_b_final_merged_s_het[['diff_b3']])
# Loop to apply the absolute value for all diff_b columns
def apply_abs_diff_b(f_b_final_merged_s_het, num_merges=3):
    for i in range(1, num_merges + 1):
        f_b_final_merged_s_het[f'diff_b{i}'] = f_b_final_merged_s_het[f'diff_b{i}'].abs()

    return f_b_final_merged_s_het

# Example usage:
f_b_final_merged_s_het = apply_abs_diff_b(f_b_final_merged_s_het, num_merges=3)

In [ ]:
f_b_final_merged_s_het

In [49]:
# Function to keep only the distance and s_het measures from the dataframe
def keep_dist_and_s_het_measures(df, num_merges=3):
    # List to store column names for distance and s_het measures
    columns_to_keep = ['variant', 'chr', 'pos']

    # Loop to collect the diff and s_het_post column names for forward and backward directions
    for i in range(1, num_merges + 1):
        columns_to_keep.append(f'diff_f{i}')
        columns_to_keep.append(f'diff_b{i}')
        columns_to_keep.append(f's_het_post_f{i}')
        columns_to_keep.append(f's_het_post_b{i}')

    # Keep only the relevant columns
    df_filtered = df[columns_to_keep]

    return df_filtered

# Example usage:
filtered_f_b_final = keep_dist_and_s_het_measures(f_b_final_merged_s_het, num_merges=3)

In [ ]:
filtered_f_b_final

In [ ]:
# Function to compute s_het weighted by 1/distance per gene-SNP pair
def compute_weighted_s_het(df, num_merges=3):
    # Loop to compute the weighted s_het for both forward and backward directions
    for i in range(1, num_merges + 1):
        # Calculate inverse distance for forward and backward directions
        df[f'inv_dist_f{i}'] = 1 / df[f'diff_f{i}']
        df[f'inv_dist_b{i}'] = 1 / df[f'diff_b{i}']
        
        # Compute weighted s_het by multiplying s_het by inverse distance
        df[f'weighted_s_het_f{i}'] = df[f'inv_dist_f{i}'] * df[f's_het_post_f{i}']
        df[f'weighted_s_het_b{i}'] = df[f'inv_dist_b{i}'] * df[f's_het_post_b{i}']

    return df

# Example usage:
weighted_f_b_final = compute_weighted_s_het(filtered_f_b_final, num_merges=3)

In [ ]:
weighted_f_b_final

In [53]:
# Function to keep only the weighted s_het values from the dataframe
def keep_weighted_s_het(df, num_merges=3):
    # List to store column names for weighted s_het values
    columns_to_keep = ['variant', 'chr', 'pos']

    # Loop to collect the weighted_s_het column names for forward and backward directions
    for i in range(1, num_merges + 1):
        columns_to_keep.append(f'weighted_s_het_f{i}')
        columns_to_keep.append(f'weighted_s_het_b{i}')

    # Keep only the relevant columns
    df_filtered = df[columns_to_keep]

    return df_filtered

# Example usage:
weighted_s_het_only = keep_weighted_s_het(weighted_f_b_final, num_merges=3)

In [ ]:
weighted_s_het_only

In [ ]:
#oliver's results 
weighted_s_het_only.iloc[np.argwhere(np.isin(weighted_s_het_only.variant, m["variant.mut"].values)).flatten()]